In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# MODEL

In [4]:
base_model = tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=(3,3),
    strides=(1, 1),
    activation='relu'
    )

inputs =  keras.Input(shape=(150, 150, 3))

base = base_model(inputs, training=False)

vectors = keras.layers.MaxPooling2D(pool_size=(2,2))(base)

flatvectors = keras.layers.Flatten()(vectors)

inner = keras.layers.Dense(64, activation='relu')(flatvectors)

outputs = keras.layers.Dense(1, activation='sigmoid')(inner)

model = keras.Model(inputs, outputs)

optimizer = keras.optimizers.SGD(learning_rate=0.002, momentum=0.8)

loss = keras.losses.BinaryCrossentropy(from_logits=False)

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['accuracy']
)

In [5]:
base.shape, vectors.shape, flatvectors.shape, inner.shape, outputs.shape

(TensorShape([None, 148, 148, 32]),
 TensorShape([None, 74, 74, 32]),
 TensorShape([None, 175232]),
 TensorShape([None, 64]),
 TensorShape([None, 1]))

**Q1. The best loss function for Binary Classification is _BinaryCrossentropy_**

In [6]:
model.count_params()

11215873

In [20]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                             

**Q2. The total number of parameters of the model = 11,215,873**

# Generators and Training

In [7]:
train_gen = ImageDataGenerator(rescale=1./255)

train_ds = train_gen.flow_from_directory(
    './cats_dogs_dataset/train',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)


Found 20000 images belonging to 2 classes.


In [8]:
val_gen = ImageDataGenerator(rescale=1./255)

val_ds = val_gen.flow_from_directory(
    './cats_dogs_dataset/validation',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

Found 5000 images belonging to 2 classes.


In [9]:
history = model.fit(train_ds,
    steps_per_epoch=100,
    epochs=10,
    validation_data=val_ds,
    validation_steps=50
)

Epoch 1/10
100/100 [==============================] - 126s 1s/step - loss: 0.6991 - accuracy: 0.5035 - val_loss: 0.6917 - val_accuracy: 0.5150
Epoch 2/10
100/100 [==============================] - 112s 1s/step - loss: 0.6892 - accuracy: 0.5350 - val_loss: 0.6867 - val_accuracy: 0.5380
Epoch 3/10
100/100 [==============================] - 126s 1s/step - loss: 0.6780 - accuracy: 0.5635 - val_loss: 0.6786 - val_accuracy: 0.5610
Epoch 4/10
100/100 [==============================] - 134s 1s/step - loss: 0.6737 - accuracy: 0.5805 - val_loss: 0.6794 - val_accuracy: 0.5540
Epoch 5/10
100/100 [==============================] - 100s 999ms/step - loss: 0.6698 - accuracy: 0.5945 - val_loss: 0.6572 - val_accuracy: 0.6110
Epoch 6/10
100/100 [==============================] - 99s 988ms/step - loss: 0.6657 - accuracy: 0.5865 - val_loss: 0.6630 - val_accuracy: 0.5970
Epoch 7/10
100/100 [==============================] - 95s 951ms/step - loss: 0.6625 - accuracy: 0.5945 - val_loss: 0.6591 - val_accuracy:

In [10]:
np.median(history.history['accuracy'])

0.590499997138977

**Q3. The median of training accuracy for this model is 0.59**

In [11]:
np.std(history.history['loss'])

0.018399237730612097

**Q4. The standard deviation of training loss for this model is 0.01**

# Data Augmentation

In [12]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_ds = train_gen.flow_from_directory(
    './cats_dogs_dataset/train',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)


Found 20000 images belonging to 2 classes.


In [13]:
history = model.fit(train_ds,
    steps_per_epoch=100,
    epochs=10,
    validation_data=val_ds,
    validation_steps=50
)

Epoch 1/10
100/100 [==============================] - 105s 1s/step - loss: 0.6695 - accuracy: 0.5815 - val_loss: 0.6403 - val_accuracy: 0.6220
Epoch 2/10
100/100 [==============================] - 106s 1s/step - loss: 0.6605 - accuracy: 0.5945 - val_loss: 0.6353 - val_accuracy: 0.6370
Epoch 3/10
100/100 [==============================] - 103s 1s/step - loss: 0.6670 - accuracy: 0.5855 - val_loss: 0.6798 - val_accuracy: 0.5660
Epoch 4/10
100/100 [==============================] - 103s 1s/step - loss: 0.6626 - accuracy: 0.5890 - val_loss: 0.6327 - val_accuracy: 0.6350
Epoch 5/10
100/100 [==============================] - 105s 1s/step - loss: 0.6671 - accuracy: 0.5975 - val_loss: 0.6355 - val_accuracy: 0.6430
Epoch 6/10
100/100 [==============================] - 104s 1s/step - loss: 0.6450 - accuracy: 0.6225 - val_loss: 0.6572 - val_accuracy: 0.5920
Epoch 7/10
100/100 [==============================] - 107s 1s/step - loss: 0.6667 - accuracy: 0.5775 - val_loss: 0.6302 - val_accuracy: 0.6500

In [14]:
np.mean(history.history['val_loss'])

0.6416960537433625

**Q5. The mean of validation loss for the model trained with augmentations is 0.64**

In [19]:
np.average(history.history['val_accuracy'][5:])

0.627400004863739

**Q6. The average of validation accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations is 0.63**